In [1]:
import streamlit as st
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.vectorstores import FAISS
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_community.embeddings import DashScopeEmbeddings
from langchain.chat_models import init_chat_model
import os

In [2]:
DeepSeek_API_KEY = os.getenv("DEEPSEEK_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")
openai_api_base = os.getenv("CHAT_API_URL")

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"




In [3]:
openai_api_base

'https://chatapi.littlewheat.com/v1'

In [4]:
from langchain.embeddings import init_embeddings
embeddings  = init_embeddings("openai:text-embedding-3-small", openai_api_key=openai_api_key, openai_api_base=openai_api_base)  # 可换本地/其他厂商

In [5]:
def pdf_read(pdf_doc):
    text = ""
    for pdf in pdf_doc:
        pdf_reader = PdfReader(pdf)
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text


def get_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = text_splitter.split_text(text)
    return chunks

def vector_store(text_chunks):
    vector_store = FAISS.from_texts(text_chunks, embedding=embeddings)
    vector_store.save_local("faiss_db")

In [6]:
def get_conversational_chain(tools, ques):
    llm = init_chat_model("deepseek-chat", model_provider="deepseek")
    prompt = ChatPromptTemplate.from_messages([
        (
            "system",
            """你是AI助手，请根据提供的上下文回答问题，确保提供所有细节，如果答案不在上下文中，请说"答案不在上下文中"，不要提供错误的答案""",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ])
    
    tool = [tools]
    agent = create_tool_calling_agent(llm, tool, prompt)
    agent_executor = AgentExecutor(agent=agent, tools=tool, verbose=True)
    
    response = agent_executor.invoke({"input": ques})
    print(response)
    st.write("🤖 回答: ", response['output'])

def check_database_exists():
    """检查FAISS数据库是否存在"""
    return os.path.exists("faiss_db") and os.path.exists("faiss_db/index.faiss")

def user_input(user_question):
    # 检查数据库是否存在
    if not check_database_exists():
        st.error("❌ 请先上传PDF文件并点击'Submit & Process'按钮来处理文档！")
        st.info("💡 步骤：1️⃣ 上传PDF → 2️⃣ 点击处理 → 3️⃣ 开始提问")
        return
    
    try:
        # 加载FAISS数据库
        new_db = FAISS.load_local("faiss_db", embeddings, allow_dangerous_deserialization=True)
        
        retriever = new_db.as_retriever()
        retrieval_chain = create_retriever_tool(retriever, "pdf_extractor", "This tool is to give answer to queries from the pdf")
        get_conversational_chain(retrieval_chain, user_question)
        
    except Exception as e:
        st.error(f"❌ 加载数据库时出错: {str(e)}")
        st.info("请重新处理PDF文件")

In [7]:
# 修改文件路径
pdf_files = ['D:/workself/project/LangChain/lcquickstart/LangChain入门.pdf'
]

# 调用函数
raw_text = pdf_read(pdf_files)
print(raw_text)


一、前言
  在当今AI应用开发领域，大型语言模型（LLM）已成为核心技术之一。然而，如
何将LLM与外部数据源、工具和API有效集成，构建高效、可扩展的智能应用，仍然
是开发者面临的挑战。为此，LangChain应运而生，提供了一个灵活、模块化的框架，帮
助开发者构建复杂的LLM应用。
  作为一个大模型开发者，在选择合适的框架时，应根据具体的应用需求、团队的技术
栈以及项目的复杂度等因素进行综合考虑。LangChain、LlamaIndex和GoogleADK
（AgentDevelopmentKit）等不同的开发框架各有优势，理解它们的特点和适用场景，
有助于做出更合适的选择。
1.LangChain

特点：提供与LLM的集成、工具调用、记忆管理、流程控制等功能，支持多种
数据源和模型的接入。


优势：模块化设计，灵活性高，适用于构建复杂的LLM应用。


适用场景：需要高度自定义和灵活性的应用，如智能客服、文档分析等。

2.LlamaIndex

特点：专注于检索增强生成（RAG）任务，提供数据加载、索引构建、查询引擎
等功能。


优势：在处理大规模数据集和高效信息检索方面表现出色。

适用场景：需要高效信息检索和问答功能的应用，如知识库构建、搜索引擎等。

3.GoogleADK

特点：提供模块化、多智能体系统的构建能力，支持结构化和动态的工作流编排。


优势：适用于构建复杂的多智能体系统，支持多种代理的协作和任务调度。


适用场景：需要多智能体协作和复杂任务调度的应用，如自动化流程、智能助手
等。

二、LangChain的背景与诞生
  LangChain由HarrisonChase于2021年提出，并于2022年作为开源项目正式发
布。其初衷是简化大型语言模型（LLM）与外部数据源、工具和API的集成，推动
LLM应用的快速开发。

2022年：功能扩展与生态起步2022年，LangChain发布了第一个版本，提供了
基础的提示词（Prompt）管理功能，并支持将工具（Tool）与语言模型结合，支持
调用外部API。同时，新增了对外部数据源的支持，包括SQL数据库、NoSQL
数据库、文件系统等，使得开发者能够将LLM与各种数据源无缝集成。

2023年：快速发展与生态构建2023年，LangC

In [8]:
text_chunks = get_chunks(raw_text)

In [9]:
vector_store(text_chunks)